project 1 - discovery

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from scipy.stats import linregress
from pprint import pprint
from sodapy import Socrata


In [ ]:
# Authenticated client request
request_url = Socrata("data.sonomacounty.ca.gov", "svXzQDyVcEYVmqw6LDWsmj7VP",
                 username="rondajoy@gmail.com", password="Flwrchild75", timeout=500)

# return as JSON from API then converted to Python list of dictionaries by sodapy
results = request_url.get("924a-vesw", limit=26000)

# Convert to pandas DataFrame
results_df = pd.DataFrame(results)
results_df.info()


---

In [ ]:
# delete unnecessary column
del results_df[':@computed_region_dig5_f3vy']
del results_df['intake_total']
del results_df['kennel_number']

# transform data type for all date columns
shelter_df['date_of_birth'] = pd.to_datetime(shelter_df.date_of_birth)
results_df['intake_date'] = pd.to_datetime(results_df.intake_date)
results_df['outcome_date'] = pd.to_datetime(results_df.outcome_date)

# Create date features for grouping
results_df['yr_intake']=results_df.intake_date.dt.year
results_df['mon_intake']=results_df.intake_date.dt.month

results_df['yr_outcome']=results_df.outcome_date.dt.year
results_df['mon_outcome']=results_df.outcome_date.dt.month

# Check for outliers via groupby year
results_df['yr_intake'].unique()
results_df.groupby('yr_intake').count()


In [ ]:
# Incomplete data for year(s) 2013 & 2023. Drop all data for those years.
results_df = results_df[(results_df['yr_intake']>2013) & (results_df['yr_intake']<2023)]

# Analysis will be heavily dependant upon "outcome type" so filter data to eliminate blank records.
# Filter out blank values by creating a new dataframe.
shelter_df = results_df.dropna(subset=['outcome_type']).copy()

# Replace missing values with either "missing" or "0" if numerical
shelter_df['name'].fillna('MISSING',inplace=True)
shelter_df['size'].fillna('MISSING',inplace=True)

# Verify successful
shelter_df.info()

In [ ]:
# Determine age on intake date
# Calculate difference between "intake_date" and "date_of_birth"
age_delta = shelter_df["intake_date"] - shelter_df["date_of_birth"]

# Convert value to numeric year(s)
age_yrs = age_delta/np.timedelta64(1, 'Y')

# Extract the number of complete months and remaining (partial) months
age_yrs = age_yrs.round(2)

shelter_df["Age at Intake (years)"] = age_yrs
shelter_df.to_csv("SoCoShelter.csv", index=False)